In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score

In [2]:
url = "https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv"
df = pd.read_csv(url)

In [3]:
# Drop columns not useful for model
df.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], axis=1, inplace=True)

# Fill missing values
df['Age'].fillna(df['Age'].median(), inplace=True)
df['Embarked'].fillna(df['Embarked'].mode()[0], inplace=True)

# Encode categorical variables
label_encoders = {}
for column in ['Sex', 'Embarked']:
    le = LabelEncoder()
    df[column] = le.fit_transform(df[column])
    label_encoders[column] = le

# Separate features and labels
X = df.drop('Survived', axis=1)
y = df['Survived']

# Normalize features
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [4]:
# Convert to PyTorch tensors
X = torch.tensor(X, dtype=torch.float32)
y = torch.tensor(y.values, dtype=torch.float32).view(-1, 1)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [5]:
class TitanicDNN(nn.Module):
    def __init__(self):
        super(TitanicDNN, self).__init__()
        self.fc1 = nn.Linear(7, 64)
        # Add a layer here of 32 neurons
        # Add a layer here of 16 neurons
        self.fc4 = nn.Linear(16, 1)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.3)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.dropout(self.relu(self.fc1(x)))
        x = self.dropout(self.relu(self.fc2(x)))
        x = self.relu(self.fc3(x))
        x = self.sigmoid(self.fc4(x))
        return x


In [6]:
model = TitanicDNN()
criterion = nn.BCELoss()  # Binary Cross Entropy
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [7]:
epochs = 100
for epoch in range(epochs):
    model.train()
    outputs = model(X_train)
    loss = criterion(outputs, y_train)
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if (epoch+1) % 10 == 0:
        print(f"Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}")

Epoch [10/100], Loss: 0.6526
Epoch [20/100], Loss: 0.6307
Epoch [30/100], Loss: 0.5957
Epoch [40/100], Loss: 0.5504
Epoch [50/100], Loss: 0.4920
Epoch [60/100], Loss: 0.4559
Epoch [70/100], Loss: 0.4495
Epoch [80/100], Loss: 0.4419
Epoch [90/100], Loss: 0.4457
Epoch [100/100], Loss: 0.4223


In [8]:
model.eval()
with torch.no_grad():
    predictions = model(X_test)
    predicted_classes = (predictions >= 0.5).float()
    acc = accuracy_score(y_test, predicted_classes)
    print(f"Test Accuracy: {acc * 100:.2f}%")

Test Accuracy: 81.01%
